# Holen aller Symbole der stockexchanges (map)

In [ ]:
import os, json
data_folder  = "/Users/a2/code/fin/trade/data/financial"
sx_symbol_map = {}
tickers = os.listdir(data_folder)
for ticker in tickers:
    with open(os.path.join(data_folder, ticker, "symbols.json"), "r") as f:
        sx_symbol_map[ticker]  = json.load(f)


In [21]:
# holen der specinal isin
import json
with open("/Users/a2/code/fin/trade/select.json", "r") as f:
    isins = json.load(f)

with open("/Users/a2/code/fin/trade/data/financial/F/d_ISIN_symbol.json", "r") as f:
    isy = json.load(f)

stocks = [isy[i] + ".F" for i in isins]

# Minutendaten

In [22]:
import yfinance as yf
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)




In [23]:
import yfinance as yf
import pandas as pd, os

import requests_cache

session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'
# ticker = yf.Ticker('msft', session=session)
# # The scraped response will be stored in the cache
# ticker.actions
data_path = "/Users/a2/code/fin/trade/data/financial/F/data/minute"

# Liste einiger LSE Symbole (du kannst diese Liste erweitern)
symbols = stocks

# Erstelle ein Dictionary, um die Daten zu speichern
stock_data = {}

# Ziehe die Minutendaten für jedes Symbol
for symbol in symbols:
    print(f"Fetching data for {symbol}...")
    ticker = yf.Ticker(symbol, session=session)
    # Hole die letzten 7 Tage Minutendaten (maximal mögliche Historie bei minutengenauer Abfrage)
    # Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
    hist = ticker.history(period='max', interval='1m')
    stock_data[symbol] = hist


for symbol, data in stock_data.items():
    # TODO: missing values padd 
    data.to_csv(os.path.join(data_path, symbol+".csv"))

print("Daten wurden erfolgreich gespeichert.")


Fetching data for ADS.F...
Fetching data for AIR.F...
Fetching data for ALV.F...
Fetching data for BAS.F...
Fetching data for BMW.F...
Fetching data for BAYN.F...
Fetching data for BEI.F...
Fetching data for BNR.F...
Fetching data for CBK.F...
Fetching data for CON.F...
Fetching data for 1COV.F...
Fetching data for DTG.F...
Fetching data for DBK.F...
Fetching data for DB1.F...
Fetching data for DHL.F...
Fetching data for P911.F...
Fetching data for DTE.F...
Fetching data for EOAN.F...
Fetching data for FRE.F...
Fetching data for HNR1.F...
Fetching data for HEI.F...
Fetching data for HEN3.F...
Fetching data for IFX.F...
Fetching data for MBG.F...
Daten wurden erfolgreich gespeichert.


In [ ]:
# hier eine etwas automatisierte stock price getter
import os
rp = "/Users/a2/code/fin/trade/data/financial"
sxsx = os.listdir(rp)
sxsx = ["F"] # temporary only for F
for sx in sxsx:
    working_dir = os.path.join(rp, sx)


In [ ]:
# Speichere die Daten in eine Excel-Datei
with pd.ExcelWriter('lse_minute_data.xlsx') as writer:
    for symbol, data in stock_data.items():
        data.to_excel(writer, sheet_name=symbol)
